In [1]:
import matplotlib.pyplot as plt
import scipy.io as sio
import skimage.io
import numpy as np
import h5py
import sys
import gc

from Data_Extractor import *

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
path_train_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_train"

# Load training set
train_set = h5py.File(path_train_set, 'r')
train_pos_topleft_coord = np.array(train_set['positive_example'])
train_neg_topleft_coord = np.array(train_set['negative_example'])
train_raw_image = np.array(train_set['raw_image'])
train_road_mask = np.array(train_set['road_mask'])
train_set.close()
gc.collect()

754

In [ ]:
path_cv_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_cv"

# Load cross-validation set
CV_set = h5py.File(path_cv_set, 'r')
CV_pos_topleft_coord = np.array(CV_set['positive_example'])
CV_neg_topleft_coord = np.array(CV_set['negative_example'])
CV_raw_image = np.array(CV_set['raw_image'])
CV_road_mask = np.array(CV_set['road_mask'])
CV_set.close()

gc.collect()

In [3]:
size = 8
norm = 'Gaussian'
Train_Data = Data_Extractor (train_raw_image, train_road_mask, size,
                             pos_topleft_coord = train_pos_topleft_coord,
                             neg_topleft_coord = train_neg_topleft_coord,
                             normalization = norm)
# run garbage collector
gc.collect()

mu =  [ 144.71187809  191.4973748   363.63933132  368.9204473  2385.57259035
 1362.40326606  728.31018062]
std =  [125.65067662 147.07994057 235.16274288 262.88676276 927.00557037
 842.7245097  545.23410243]


20

In [ ]:
size = 8
norm = 'Gaussian'
CV_Data = Data_Extractor (CV_raw_image, CV_road_mask, size,
                          pos_topleft_coord = CV_pos_topleft_coord,
                          neg_topleft_coord = CV_neg_topleft_coord,
                          normalization = norm)
gc.collect()

In [ ]:
def is_close_to_edge(coord, raw_img, size):
    dist = int(1.5*size)
    center = coord + int(size/2)
    return (raw_img[0,center[0]-dist:center[0]+dist, center[1]-dist:center[1]+dist] == -9999).any()

In [ ]:
h5f = h5py.File('./Result/center_edge_img_train.h5', 'w')
h5f.create_group(name='edge', data=edge_img)
h5f.create_group(name='center', data=center_img)
h5f.close()

for ch_n in range(7):
    edge_img = []
    center_img = []
    i = 0
    for coord, patch, y in Train_Data.iterate_data_with_coord(norm=True):
        patch = patch[0]
        if is_close_to_edge(coord,  train_raw_image, size):
            edge_img.extend(patch[ch_n])
        else:
            center_img.extend(patch[ch_n])
        
        i += 1
        if i > 1000:
            gc.collect()
            i = 0
    gc.collect()
    
    h5f = h5py.File('./Result/center_edge_img_train.h5', 'w')
    edge_group = h5f['edge']
    edge_group.create_dataset(name=str(ch_n), data=edge_img)

    gc.collect()

    center_group = h5f['center']
    center_group.create_dataset(name=str(ch_n), data=center_img)

    gc.collect()

In [ ]:
h5f = h5py.File('./Result/center_edge_img_train.h5', 'w')
h5f.create_dataset(name='edge', data=edge_img)
h5f.create_dataset(name='center', data=center_img)
h5f.close()

In [ ]:
# Two subplots, unpack the axes array immediately
for ch_n in range(7):


In [ ]:
gc.collect()

In [ ]:
# Two subplots, unpack the axes array immediately
for ch_n in range(7):
    plt.figure(figsize=(30,30))
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    cur_img = edge_img[:,0,:,:]
    ax1.hist(cur_img[np.where(cur_img>0)].flatten(), bins=150,  histtype='step')
    ax1.set_title('edge_' + str(ch_n))
    gc.collect()
    
    cur_img = center_img[:,0,:,:]
    ax2.hist(cur_img[np.where(cur_img>0)].flatten(), bins=150,  histtype='step')
    ax1.set_title('center_' + str(ch_n))
    plt.grid(True)
    plt.show()
    plt.close()
    gc.collect()

In [ ]:
class FCN_Data_Extractor (Data_Extractor):

    def __init__(self, raw_image, road_mask, img_size, pos_topleft_coord, neg_topleft_coord, normalization='mean', encoding='one-hot'):

        super(FCN_Data_Extractor, self).__init__(raw_image, road_mask, img_size, pos_topleft_coord, neg_topleft_coord, normalization, encoding)
        self.neg_weight = self.pos_size / self.size
        self.pos_weight = self.neg_size / self.size
        
    def _get_patch_label(self, coord):
        label = self.road_mask[coord[0]:coord[0]+self.img_size, coord[1]:coord[1]+self.img_size]
        if self.encoding == 'one-hot':
            one_hot_label = np.zeros((self.img_size, self.img_size, 2))
            one_hot_label[:, :, 0][np.where(label == 0)] = 1
            one_hot_label[:, :, 1][np.where(label == 1)] = 1
            return one_hot_label
        return label

    def get_patches(self, batch_size, positive_num = 0, norm = True, wrap_around=True, weighted=True):
        X = []
        Y = []

        if positive_num > 0 and positive_num <= batch_size:
            # pos patches
            X_pos, Y_pos, self.pos_index = self._get_patches_from_topleft_coord (self.pos_topleft_coord,
                                                                     num_of_patches = positive_num,
                                                                     start_index = self.pos_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            negative_num = batch_size - positive_num
            X_neg, Y_neg, self.neg_index = self._get_patches_from_topleft_coord (self.neg_topleft_coord,
                                                                     num_of_patches = negative_num,
                                                                     start_index = self.neg_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            X.extend (X_pos)
            Y.extend (Y_pos)
            
            X.extend (X_neg)
            Y.extend (Y_neg)
        
        else: # sample batches randomly
            X, Y, self.index = self._get_patches_from_topleft_coord (self.topleft_coordinate,
                                                                      num_of_patches = batch_size,
                                                                      start_index = self.index,
                                                                      norm = norm,
                                                                      wrap_around=wrap_around)

        X = np.array(X)
        Y = np.array(Y)
    
        if weighted:
            Weight = Y.copy()
            Weight[:,:,:,0] *= self.neg_weight
            Weight[:,:,:,1] *= self.pos_weight
            Weight = Weight.sum(axis=-1)

        return X, Y, Weight

In [ ]:
import h5py
import gc

path_train_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_train"
path_cv_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_cv"

# Load training set
train_set = h5py.File(path_train_set, 'r')
train_pos_topleft_coord = np.array(train_set['positive_example'])
train_neg_topleft_coord = np.array(train_set['negative_example'])
train_raw_image = np.array(train_set['raw_image'])
train_road_mask = np.array(train_set['road_mask'])
train_set.close()

# Load cross-validation set
CV_set = h5py.File(path_cv_set, 'r')
CV_pos_topleft_coord = np.array(CV_set['positive_example'])
CV_neg_topleft_coord = np.array(CV_set['negative_example'])
CV_raw_image = np.array(CV_set['raw_image'])
CV_road_mask = np.array(CV_set['road_mask'])
CV_set.close()

gc.collect()

In [ ]:
X, Y = CV_Data.get_patches(2)
print(X.shape, Y.shape)

In [ ]:
CV_FCN_Data = FCN_Data_Extractor (CV_raw_image, CV_road_mask, 8,
                                  pos_topleft_coord = CV_pos_topleft_coord,
                                  neg_topleft_coord = CV_neg_topleft_coord)

In [ ]:
x, y, w = CV_FCN_Data.get_patches(2)
print(x.shape, y.shape, w.shape)

In [ ]:

y_f = y.reshape(-1,2)
w_f = w.reshape(-1)
for i in range(y_f.shape[0]):
    print(y_f[i], w_f[i])
    assert sum(np.array([CV_FCN_Data.neg_weight, CV_FCN_Data.pos_weight]) * y_f[i]) == w_f[i] 